In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
# %config InlineBackend.figure_format = 'svg'

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
os.environ['THEANO_FLAGS'] = "device=cpu"

In [ ]:
import os
import shelve
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['savefig.dpi'] = 300

from experiments.hmmvsrnn_reco.a_data import tmpdir

In [ ]:
report_files = [os.path.join(d, s[:-4])
                for d in [tmpdir, tmpdir + '.run2']
                for s in os.listdir(d) if s.endswith(".dat")]

analyses = []

for f in report_files:
    with shelve.open(f, flag='r') as report:
        if 'analysis' not in report.keys():
            continue
        recap = report['analysis']
        args = report['args']['encoder_kwargs']
        meta = report['meta']
        analyses.append((meta, args, recap))
#         print("->", meta['experiment_name'])
#         print("Accuracy:   {:.4f} / {:.4f} / ?".format(*recap['accuracy'][:2]))
#         print("JI:         {:.4f} / {:.4f} / ?".format(*recap['ji'][:2]))
#         print("Acc. filt.:      ? / {:.4f} / {:.4f}".format(*recap['accuracy_filtered'][1:]))
#         print("JI filt.:        ? / {:.4f} / {:.4f}".format(*recap['ji_filtered'][1:]))

dtype = [('model', 'S3'), 
         ('win', 'i'), ('nparms', 'i'),
         ('tc', 'i'), ('num_tc_filters', 'i'), 
         ('ji', 'f'), ('acc', 'f')]

analyses = np.array([
    np.array((
        m['model'], 
        a['filter_dilation'] * (a['tconv_sz'] - 1) + 1, a['tconv_sz'] * a['num_tc_filters'],
        a['tconv_sz'], a['num_tc_filters'],
        r['ji_filtered'][1], 
        r['accuracy_filtered'][1]), 
        dtype=dtype)
    for m, a, r in analyses if m['modality'] == 'skel'])

analyses = np.sort(analyses)
print(analyses)

In [ ]:
plt.figure(dpi=100) 

legend = []
subset = (analyses['model'] == b"rnn")
p1 = plt.scatter(analyses[subset]['win'], analyses[subset]['ji'], s=analyses[subset]['nparms'] / 50, marker="o", alpha=0.5)
    
legend = []
subset = (analyses['model'] == b"hmm")
p2 = plt.scatter(analyses[subset]['win'], analyses[subset]['ji'], s=analyses[subset]['nparms'] / 50, marker="o", alpha=0.5)

plt.legend([p1, p2], ['rnn', 'hmm'], loc='lower right')

plt.xlabel("window size")
plt.ylabel("Jaccard Index")
plt.xticks(np.arange(3, 32, 4))
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.savefig('blah.png')
plt.show()

In [ ]:
# compare confusion on video frames

with shelve.open(os.path.join(tmpdir, "hmm_bgr_tc15"), flag='r') as report:
    hmm_meta = report['meta']
    hmm_args = report['args']
    hmm_analysis = report['analysis']
    
with shelve.open(os.path.join(tmpdir, "rnn_bgr_tc15"), flag='r') as report:
    rnn_meta = report['meta']
    rnn_args = report['args']
    rnn_analysis = report['analysis']

plt.figure(dpi=150)

bgr_conf = hmm_analysis['confusion_filtered'][1] / np.sum(hmm_analysis['confusion_filtered'][1], axis=1)
rnn_conf = rnn_analysis['confusion_filtered'][1] / np.sum(rnn_analysis['confusion_filtered'][1], axis=1)
conf_diff = bgr_conf - rnn_conf
limits = np.max(abs(conf_diff))
plt.imshow(conf_diff, clim=(-limits, limits), cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.4, 0.1), xycoords='axes fraction', xytext=(1.4, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.37, 0.05), xycoords='axes fraction', xytext=(1.37, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.34, 0.92), xycoords='axes fraction', xytext=(1.34, 0.92))
plt.colorbar()

a = np.sum(hmm_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(hmm_analysis['confusion_filtered'][1][1:, 1:]))
b = np.sum(rnn_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(rnn_analysis['confusion_filtered'][1][1:, 1:]))
print(a, b, (a - b) / b)

In [ ]:
# compare confusion on body poses

with shelve.open(os.path.join(tmpdir, "hmm_skel_tc15"), flag='r') as report:
    hmm_meta = report['meta']
    hmm_args = report['args']
    hmm_analysis = report['analysis']
    
with shelve.open(os.path.join(tmpdir, "rnn_skel_tc15"), flag='r') as report:
    rnn_meta = report['meta']
    rnn_args = report['args']
    rnn_analysis = report['analysis']
    
plt.figure(dpi=150)

bgr_conf = hmm_analysis['confusion_filtered'][1] / np.sum(hmm_analysis['confusion_filtered'][1], axis=1)
rnn_conf = rnn_analysis['confusion_filtered'][1] / np.sum(rnn_analysis['confusion_filtered'][1], axis=1)
conf_diff = bgr_conf - rnn_conf
limits = np.max(abs(conf_diff))
plt.imshow(conf_diff, clim=(-limits, limits), cmap='RdBu')
plt.yticks(np.arange(0, 21), [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo'],
    fontsize=8)
plt.xticks(np.arange(0, 21), [''] * 21)
plt.gca().annotate(
    '', 
    xy=(1.4, 0.1), xycoords='axes fraction', xytext=(1.4, 0.9), 
    arrowprops=dict(arrowstyle="<->", color='k'))
plt.gca().annotate(
    'rnn', xy=(1.37, 0.05), xycoords='axes fraction', xytext=(1.37, 0.05))
plt.gca().annotate(
    'hmm', xy=(1.34, 0.92), xycoords='axes fraction', xytext=(1.34, 0.92))
plt.colorbar()

a = np.sum(hmm_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(hmm_analysis['confusion_filtered'][1][1:, 1:]))
b = np.sum(rnn_analysis['confusion_filtered'][1][1:, 1:]) - np.sum(np.diag(rnn_analysis['confusion_filtered'][1][1:, 1:]))
print(a, b, (a - b) / b)

In [ ]:
# from sklearn.manifold import TSNE
# from sltools.tconv import TemporalConv

# tc_l = None
# layers = lasagne.layers.get_all_layers(model_dict['l_linout'])

# for l in layers:
#     if isinstance(l, TemporalConv):
#         tc_l = l
#         break

# W = np.asarray(tc_l.W.eval())
# tsne = TSNE(n_components=1, n_iter=5000, n_iter_without_progress=100, verbose=True)
# filter_order = np.argsort(tsne.fit_transform(W)[:, 0])